# Causal Demand Shock Attribution

This notebook explains the root cause behind detected demand anomalies by
distinguishing between:
- Trend-driven changes
- Seasonal fluctuations
- External demand shocks

The goal is to move beyond anomaly detection and provide actionable,
explainable insights for decision-makers.


In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("../data/causal_features.csv", parse_dates=["date"])
df.set_index("date", inplace=True)
df.head()


,demand,event_type,intensity,demand_lag_1,demand_lag_7,demand_lag_14,rolling_mean_7,rolling_std_7,rolling_mean_14,rolling_std_14,z_score_7,trend,seasonal,residual,trend_change,shock_strength
date,,,,,,,,,,,,,,,,
2022-01-15,46.807708,none,0.0,45.805035,55.019450,52.158990,50.526539,3.334374,51.975048,3.550438,-1.115301,50.766372,-0.758133,-3.200531,-0.001155,3.200531
2022-01-16,51.706695,none,0.0,46.807708,50.324531,49.873576,50.723991,3.361233,52.105986,3.500426,0.292364,50.971607,-0.547624,1.282712,0.205235,1.282712
2022-01-17,50.152181,none,0.0,51.706695,54.624930,53.271786,50.085027,2.887879,51.883157,3.519745,0.023254,51.427850,-1.029004,-0.246665,0.456243,0.246665
2022-01-18,55.707300,none,0.0,50.152181,50.852772,57.027396,50.778531,3.598448,51.788864,3.386492,1.369693,52.057217,2.084786,1.565297,0.629367,1.565297
2022-01-19,51.064117,none,0.0,55.707300,51.094719,50.252705,50.774159,3.598018,51.846822,3.365055,0.080588,52.759686,-0.356731,-1.338838,0.702469,1.338838


In [5]:
df.columns


Index(['demand', 'event_type', 'intensity', 'demand_lag_1', 'demand_lag_7',
       'demand_lag_14', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14',
       'rolling_std_14', 'z_score_7', 'trend', 'seasonal', 'residual',
       'trend_change', 'shock_strength'],
      dtype='object')

In [7]:
def attribute_cause(row):
    if "trend_change" in row and abs(row["trend_change"]) > 0.15:
        return "Trend Shift"
    elif "z_score_7" in row and abs(row["z_score_7"]) > 3:
        return "External Shock"
    elif "rolling_std_7" in row and row["rolling_std_7"] > df["rolling_std_7"].median():
        return "Seasonal Volatility"
    else:
        return "Normal Variation"


In [8]:
df["shock_cause"] = df.apply(attribute_cause, axis=1)
df[["demand", "shock_cause"]].head()


,demand,shock_cause
date,,
2022-01-15,46.807708,Normal Variation
2022-01-16,51.706695,Trend Shift
2022-01-17,50.152181,Trend Shift
2022-01-18,55.707300,Trend Shift
2022-01-19,51.064117,Trend Shift


In [9]:
df.reset_index().to_csv("../data/causal_features.csv", index=False)
print("Saved causal_features.csv with shock_cause column")


Saved causal_features.csv with shock_cause column
